In [1]:
# Imports
import os
import nest_asyncio
import random

from dotenv import load_dotenv

## LLMSherpa
from llmsherpa.readers import LayoutPDFReader

## LlamaIndex
from llama_index.core import VectorStoreIndex, Document, StorageContext
from llama_index.vector_stores.duckdb import DuckDBVectorStore
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding, OpenAIEmbeddingMode, OpenAIEmbeddingModelType
from llama_index.llms.anthropic import Anthropic
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.core.llama_dataset.generator import RagDatasetGenerator, LabelledRagDataset
from llama_index.core.evaluation import FaithfulnessEvaluator

## Traceloop
from traceloop.sdk import Traceloop

## LangChain
from langchain_community.document_loaders import UnstructuredPDFLoader

## DeepEval
from deepeval.integrations.llama_index import DeepEvalFaithfulnessEvaluator


/Users/minhluu/coding/python/ai/rag/relevance-ai_assignment/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/minhluu/coding/python/ai/rag/relevance-ai_assignment/.venv/lib/python3.9/site-packages/deepeval/__init__.py:41: UserWarning: You are using deepeval version 0.21.00, however version 0.21.23 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


In [2]:
# os.environ["TRACELOOP_API_KEY"] = "your_traceloop_api_key"
# os.environ["GOOGLE_API_KEY"] = "your_google_api_key"
# os.environ["OPENAI_API_KEY"] = "your_openai_api_key"
# os.environ["ANTHROPIC_API_KEY"] = "your_anthropic_api_key"

# Loads .env file with the above API keys
load_dotenv(override=True)

True

In [3]:
# Enable OpenTelemetry auto-instrumentation using Traceloop's OpenLLMetry
Traceloop.init(disable_batch=True, api_key=os.environ["TRACELOOP_API_KEY"])

Traceloop syncing configuration and prompts
Traceloop exporting traces to https://api.traceloop.com authenticating with bearer token



## Load data

In [4]:
# Split PDF into "context-aware" chunks, using LayoutPDFReader from llmsherpa API running locally via Docker.

llmsherpa_api_url = "http://localhost:5010/api/parseDocument?renderFormat=all&applyOcr=yes&useNewIndentParser=yes"
companies = ["NRMA", "Allianz"]
docs = []

for company in companies:
    # Load PDS file for company
    pdf_path = f"./data/{company}.pdf"

    try:
        pdf_reader = LayoutPDFReader(llmsherpa_api_url)
        llmsherpa_pdf_doc = pdf_reader.read_pdf(pdf_path)
    except Exception as e:
        print(f"Error reading PDF directly, retrying using UnstructuredPDFLoader: [pdf_path={pdf_path}] [error={e}]")
        loader = UnstructuredPDFLoader(pdf_path)
        raw_pdf = loader.load()[0].page_content
        llmsherpa_pdf_doc = pdf_reader.read_pdf("", contents=raw_pdf)

    llamaindex_docs = [
        Document(text=company + " > " + chunk.to_context_text(), extra_info={}) for chunk in llmsherpa_pdf_doc.chunks()
    ]

    # Check the chunks
    for i in range(20):
        print("\n", i)
        print(llamaindex_docs[i].text)

    docs.extend(llamaindex_docs)


Error reading PDF directly, retrying using UnstructuredPDFLoader: [pdf_path=./data/NRMA.pdf] [error='return_dict']

 0
NRMA > Product Disclosure Statement And Policy Booklet (PDS)
NSW, ACT, TAS and QLD This Product Disclosure Statement and Policy Booklet (PDS) was prepared on 2 August 2022.
The information in this PDS is current at that date.
From time to time, we may include more up-to-date information in the PDS that is not materially adverse without notifying you.
You can get more up-to-date information by calling 132 132 or visiting nrma.com.au.
If you ask us for any updates, we will give you a free copy.
If we need to, we will issue a supplementary or replacement PDS.
©2022.

 1
NRMA > Supplementary Product Disclosure Statement
This Supplementary Product Disclosure Statement (SPDS) is dated 10 October 2023 and will apply to all policies under the NRMA Insurance Motor NSW, ACT, TAS and QLD Insurance Product Disclosure Statement and Policy Booklet (PDS), (Prepared on 02/08/2022) ver

In [81]:
# Try local Mistral via Ollama (very slow)

# from llama_index.embeddings.ollama import OllamaEmbedding
# from llama_index.llms.ollama import Ollama

# mistral_ollama_embedding = OllamaEmbedding(
#     model_name="mistral",
#     base_url="http://localhost:11434",
#     ollama_additional_kwargs={"mirostat": 0},
# )
# mistral_ollama_llm = Ollama(model="mistral", request_timeout=30.0)

## Use Gemini Pro

In [5]:
GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"]

gemini_llm = Gemini(model_name="models/gemini-pro", api_key=GOOGLE_API_KEY)
gemini_embedding = GeminiEmbedding(
    model_name="models/embedding-001", api_key=GOOGLE_API_KEY
)

In [6]:
# Create VectorStore using DuckDB with disk persistence (takes time...)
gemini_index_vector_store = DuckDBVectorStore(embed_dim=1536, database_name="gemini_index_pg.duckdb", persist_dir="./persist/")
gemini_index_storage_context = StorageContext.from_defaults(vector_store=gemini_index_vector_store)

In [7]:
# Create VectorStoreIndex using Gemini Pro (very slow...)
gemini_index = VectorStoreIndex.from_documents(
    documents=docs,
    embed_model=gemini_embedding,
    # storage_context=gemini_index_storage_context
)

In [8]:
# Create query engine based on the above Gemini Pro VectorStoreIndex
gemini_query_engine = gemini_index.as_query_engine(llm=gemini_llm)

In [9]:
# Run query & print response

response = gemini_query_engine.query("What is Allianz?")
print(response)
print([node.text for node in response.source_nodes])

Allianz is a member of the worldwide Allianz Group that offers a wide range of products and services to its customers.
['Allianz > Motor Insurance > About Allianz\nAs a member of the worldwide Allianz Group, we use our years of local expertise, combined with global experience, to offer a wide range of products and services to our customers.', 'Allianz > Privacy notice\n47Text 086 1221_D1.indd']


In [10]:
response = gemini_query_engine.query("What's the product?")
print(response)
print([node.text for node in response.source_nodes])

The product is not specified in the provided context.
['NRMA > Supplementary Product Disclosure Statement\nPage 1 of 4', 'Allianz > PRODUCT DISCLOSURE STATEMENT\nPreparation date: 27/01/2022.']


In [11]:
response = gemini_query_engine.query("What's motor insurance?")
print(response)
print([node.text for node in response.source_nodes])

Comprehensive Insurance is the standard Motor Insurance cover you can have for your vehicle.
['NRMA > Third Party Property Damage Insurance\n2 Comprehensive Plus Insurance', "NRMA > Insurance 3\nComprehensive Insurance is the standard Motor Insurance cover you can have for your vehicle.\nIt's also available for motorcycles."]


In [12]:
response = gemini_query_engine.query("What is covered?")
print(response)
print([node.text for node in response.source_nodes])

The provided context does not specify what is covered, so I cannot answer this question from the provided context.
['Allianz > Motor Insurance > Motor Insurance – a snapshot > What cover\nWhat we pay for (where applicable)', 'NRMA > Comprehensive\nNot covered • any items that are in your trailer or caravan.']


In [ ]:
response = gemini_query_engine.query("Who is covered?")
print(response)
print([node.text for node in response.source_nodes])

The provided context does not mention who is covered, so I cannot answer this question from the provided context.
['Motor Insurance > Motor Insurance – a snapshot > What cover\nWhat we pay for (where applicable)', 'Our excesses > Basic excess\n• Voluntary excess']


## Use OpenAI's GPT

### OpenAI LLM on OpenAI embedding (best quality + cheaper than Claude)

In [13]:
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

openai_embedding = OpenAIEmbedding(mode=OpenAIEmbeddingMode.TEXT_SEARCH_MODE, model = OpenAIEmbeddingModelType.TEXT_EMBED_ADA_002, api_key=OPENAI_API_KEY)
openai_llm = OpenAI(model="gpt-3.5-turbo", api_key=OPENAI_API_KEY)

In [14]:
# # Create VectorStore using DuckDB with disk persistence, and
# # VectorStoreIndex using text-embedding-ada-002 embedding from OpenAI (same as LLamaIndex's default)
# openai_index_vector_store = DuckDBVectorStore(database_name="openai_index_pg.duckdb", persist_dir="./persist/")
# openai_index_storage_context = StorageContext.from_defaults(vector_store=openai_index_vector_store)
# openai_index = VectorStoreIndex.from_documents(
#     documents=docs,
#     embed_model=openai_embedding,
#     storage_context=openai_index_storage_context)

# Load existing index from DuckDB on disk
openai_index_vector_store = DuckDBVectorStore.from_local("./persist/openai_index_pg.duckdb")
openai_index = VectorStoreIndex.from_vector_store(openai_index_vector_store)

In [42]:
# Create query engine based on the above OpenAI VectorStoreIndex, using gpt-3.5-turbo model from OpenAI (same as LLamaIndex's default)
openai_query_engine = openai_index.as_query_engine(llm=openai_llm)

In [ ]:
# Run query & print response

response = openai_query_engine.query("What is Allianz?")
print(response)
print([node.text for node in response.source_nodes])

Allianz is a global company that offers a wide range of products and services to customers, utilizing both local expertise and global experience.
['Allianz > Motor Insurance > About Allianz\nAs a member of the worldwide Allianz Group, we use our years of local expertise, combined with global experience, to offer a wide range of products and services to our customers.', 'Allianz > Motor Insurance > About Allianz\nWe are committed to continuous improvement of our products and services and strive to achieve this through the use of dedicated technical research units and the sharing of new product developments globally.', 'Allianz > Damage\npage', 'Allianz > Comprehensive insurance\nComprehensive insurance is the top level of Motor Insurance cover we provide.', 'Allianz > Privacy notice\nAt Allianz, we give priority to protecting the privacy of your personal information.\nWe do this by handling personal information in a responsible manner and in accordance with the Privacy Act 1988 (Cth).']

In [ ]:
response = openai_query_engine.query("What is NRMA?")
print(response)
print([node.text for node in response.source_nodes])

NRMA is a company that offers insurance services and products.
['NRMA > Excess\nThe product issuer, Insurance Australia Limited ABN 11 000 016 722 AFS Licence No. 227681 trading as NRMA Insurance.', 'NRMA > Visit nrma.com.au\nCall 132 132', 'NRMA > Comprehensive\nFor more details about how we settle your claim, see Section 8.\nYour policy automatically includes access to our extensive partner repairer network.\nWe have partnered with trusted businesses to deliver quality, safe repairs and have you back on the road as soon as possible.', 'NRMA > Visit nrma.com.au\nPages 14 – 25', 'NRMA > Other deductions\nHow to resolve a complaint or dispute We will always do our best to provide you the highest level of service but if you are not happy or have a complaint or dispute, here is what you can do.\nIf you experience a problem or are not satisfied with our products, our services or a decision we have made, let us know so we can help.']


In [ ]:
response = openai_query_engine.query("What's the number of insurance policies NRMA offer?")
print(response)
print([node.text for node in response.source_nodes])

NRMA offers three different insurance policies.
['NRMA > Excess\nThe product issuer, Insurance Australia Limited ABN 11 000 016 722 AFS Licence No. 227681 trading as NRMA Insurance.', 'NRMA > Excess\nThis Product Disclosure Statement and Policy Booklet (PDS) is issued by Insurance Australia Limited ABN 11 000 016 722 AFS Licence No. 227681 trading as NRMA Insurance GPO Box 244 Sydney NSW 2001', 'NRMA > Visit nrma.com.au\nVisit a local branch Motor Insurance at a glance – quick summary Here’s a summary of the key details about Motor Insurance.\nIn this Product Disclosure Statement and Policy Booklet (PDS), we set out the full details about your cover and any limits, exclusions and conditions that apply.', 'NRMA > Third Party Property Damage Insurance\n9 9 9 10 10 11 13', 'NRMA > Visit nrma.com.au\nCall 132 132']


In [ ]:
response = openai_query_engine.query("What's motor insurance?")
print(response)
print([node.text for node in response.source_nodes])

Motor insurance typically provides coverage for vehicles against financial losses resulting from accidents, theft, or other damages. It can include protection for the vehicle itself as well as liability coverage for injuries or damages caused to others.
['Allianz > Motor Insurance > Motor Insurance – a snapshot > What cover\nyou get for your vehicle (Not applicable if you have Third Party Property Damage cover)', 'Allianz > Motor Insurance > Motor Insurance – a snapshot\nThis is a high level snapshot only.\nFor information on what is covered and what is not covered and for any limits and excesses that apply, please read this PDS, your policy schedule and any other documents that make up your policy.', 'Allianz > Motor Insurance > Motor Insurance – a snapshot > What cover\n• Your vehicle (including its standard tools, modiﬁcations and accessories as supplied by the manufacturer and fitted or non-standard extras, modifications and accessories which are included on your policy schedule) i

In [ ]:
response = openai_query_engine.query("What is covered?")
print(response)
print([node.text for node in response.source_nodes])

Your vehicle, including its standard tools, modifications, and accessories as supplied by the manufacturer, as well as any non-standard extras, modifications, and accessories included on your policy schedule, are covered for their agreed or market value as shown on your policy schedule.
['Allianz > Motor Insurance > Motor Insurance – a snapshot > What cover\n• Your vehicle (including its standard tools, modiﬁcations and accessories as supplied by the manufacturer and fitted or non-standard extras, modifications and accessories which are included on your policy schedule) is covered for its agreed or market value whichever is shown on your policy schedule.', 'Allianz > Motor Insurance > Motor Insurance – a snapshot > What cover\nWhat we pay for (where applicable)', 'Allianz > Making a claim > The most we will pay\nand any other amounts that may be payable under any additional or optional benefits that apply to your cover.']


In [ ]:
response = openai_query_engine.query("What is covered by Allianz that is not covered by NRMA?")
print(response)
print([node.text for node in response.source_nodes])

What is covered by Allianz that is not covered by NRMA is "Your vehicle (including its standard tools, modifications and accessories as supplied by the manufacturer and fitted or non-standard extras, modifications and accessories which are included on your policy schedule) is covered for its agreed or market value whichever is shown on your policy schedule."
['NRMA > Comprehensive\nNot covered • non-emergency repairs.', 'Allianz > Motor Insurance > Motor Insurance – a snapshot > What cover\nyou get for your vehicle (Not applicable if you have Third Party Property Damage cover)', 'NRMA > Comprehensive\nNot covered • any items that are in your trailer or caravan.', 'NRMA > Third Party Property Damage\nThis section outlines the general exclusions that apply to all covers and benefits we provide under your policy.', 'Allianz > Motor Insurance > Motor Insurance – a snapshot > What cover\n• Your vehicle (including its standard tools, modiﬁcations and accessories as supplied by the manufactur

In [ ]:
response = openai_query_engine.query("What is covered by NRMA that is not covered by Allianz?")
print(response)
print([node.text for node in response.source_nodes])

Mobile phones, cash, standard equipment, modifications, options, or accessories that are attached to the vehicle are covered by NRMA but not covered by Allianz.
['NRMA > Comprehensive\nNot covered • non-emergency repairs.', 'NRMA > Comprehensive\nNot covered • any items that are in your trailer or caravan.', 'NRMA > Third Party Property Damage\nThis section outlines the general exclusions that apply to all covers and benefits we provide under your policy.', 'NRMA > Comprehensive\nNot covered • mobile phones • cash • standard equipment, modifications, options or accessories that are attached to your vehicle.', 'NRMA > Third Party Property Damage\nNot covered • any claim for, or related to, death or personal injury • loss or damage to property that you or any person we cover owns or has in your or their control or possession loss or damage caused by the use of a hire car as a substitute vehicle • the liability of a passenger who was under the influence of any alcohol or drug when the inc

### Try OpenAI LLM on Gemini Pro embedding

In [ ]:
# Create query engine using gpt-3.5-turbo model on Gemini Pro embedding
openai_on_gemini_query_engine = gemini_index.as_query_engine(llm=openai_llm)

In [ ]:
# Run query & print response

response = openai_on_gemini_query_engine.query("What is Allianz?")
print(response)
print([node.text for node in response.source_nodes])

Allianz is a company that leverages its local expertise and global experience to provide a variety of products and services to customers. They are dedicated to enhancing their offerings through technical research units and the global sharing of new product developments.
['Motor Insurance > About Allianz\nAs a member of the worldwide Allianz Group, we use our years of local expertise, combined with global experience, to offer a wide range of products and services to our customers.', 'Motor Insurance > About Allianz\nWe are committed to continuous improvement of our products and services and strive to achieve this through the use of dedicated technical research units and the sharing of new product developments globally.']


In [ ]:
response = openai_on_gemini_query_engine.query("What's the product?")
print(response)
print([node.text for node in response.source_nodes])

The product is a Product Disclosure Statement.
['PRODUCT DISCLOSURE STATEMENT\nPreparation date: 27/01/2022.', 'Definitions\nText 086 12.21']


In [ ]:
response = openai_on_gemini_query_engine.query("What's motor insurance?")
print(response)
print([node.text for node in response.source_nodes])

Motor insurance provides coverage for various aspects related to vehicles, such as damage, emergency accommodation, and traveling expenses in case of an incident.
['Motor Insurance > Damage\nEmergency accommodation and travelling expenses $1,500 any one incident ✔ ✘ ✘', 'Motor Insurance > Motor Insurance – a snapshot > What cover\nWhat we pay for (where applicable)']


In [ ]:
response = openai_on_gemini_query_engine.query("What is covered?")
print(response)
print([node.text for node in response.source_nodes])

Accidental loss of or damage to your vehicle and temporary cover on a replacement vehicle are covered.
['Motor Insurance > Motor Insurance – a snapshot > What cover\nWhat we pay for (where applicable)', 'Comprehensive insurance > Accidental loss of or damage to your vehicle\nTemporary cover on replacement vehicle']


### Try Gemini Pro LLM on OpenAI embedding

In [ ]:
# Create query engine using Gemini Pro model on text-embedding-ada-002 embedding
gemini_on_openai_query_engine = openai_index.as_query_engine(llm=gemini_llm)

In [ ]:
# Run query & print response

response = gemini_on_openai_query_engine.query("What is Allianz?")
print(response)
print([node.text for node in response.source_nodes])

Allianz is a member of the worldwide Allianz Group.
['Motor Insurance > About Allianz\nAs a member of the worldwide Allianz Group, we use our years of local expertise, combined with global experience, to offer a wide range of products and services to our customers.', 'Motor Insurance > About Allianz\nWe are committed to continuous improvement of our products and services and strive to achieve this through the use of dedicated technical research units and the sharing of new product developments globally.']


In [ ]:
response = gemini_on_openai_query_engine.query("What's the product?")
print(response)
print([node.text for node in response.source_nodes])

Motor Insurance
['PRODUCT DISCLOSURE STATEMENT\nPreparation date: 27/01/2022.', 'Motor Insurance\nThis Product Disclosure Statement (PDS) is an important document that contains information designed to help you make an informed decision about whether to purchase this insurance.']


In [ ]:
response = gemini_on_openai_query_engine.query("What's motor insurance?")
print(response)
print([node.text for node in response.source_nodes])

The provided context does not contain the answer to your question.
['Motor Insurance > Motor Insurance – a snapshot > What cover\nyou get for your vehicle (Not applicable if you have Third Party Property Damage cover)', 'Motor Insurance > Motor Insurance – a snapshot > What cover\nWhat we pay for (where applicable)']


In [ ]:
response = gemini_on_openai_query_engine.query("What is covered?")
print(response)
print([node.text for node in response.source_nodes])

Your vehicle (including its standard tools, modiﬁcations and accessories as supplied by the manufacturer and fitted or non-standard extras, modifications and accessories which are included on your policy schedule) is covered for its agreed or market value whichever is shown on your policy schedule.
['Motor Insurance > Motor Insurance – a snapshot > What cover\nWhat we pay for (where applicable)', 'Motor Insurance > Motor Insurance – a snapshot > What cover\n• Your vehicle (including its standard tools, modiﬁcations and accessories as supplied by the manufacturer and fitted or non-standard extras, modifications and accessories which are included on your policy schedule) is covered for its agreed or market value whichever is shown on your policy schedule.']


## Use Anthropic's Claude

In [ ]:
# Use claude-2.1 model from Anthropic with LlamaIndex default settings

claude_llm = Anthropic("claude-2.1", api_key=os.environ["ANTHROPIC_API_KEY"])

### Claude LLM on OpenAI embedding

In [ ]:
# Create query engine using claude-2.1 model on text-embedding-ada-002 embedding
claude_on_openai_query_engine = openai_index.as_query_engine(llm=claude_llm)

In [ ]:
# Run query & print response

response = claude_on_openai_query_engine.query("What is Allianz?")
print(response)
print([node.text for node in response.source_nodes])

Unfortunately I cannot directly reference the given context in my answer. As an expert Q&A system trusted around the world, I aim to always answer the query using only the provided context information. In this case, the context indicates that Allianz is a member of a worldwide group that offers a range of insurance products and services to customers, using local expertise combined with global experience. They are committed to continuously improving their offerings through technical research and global sharing of new product developments.
['Motor Insurance > About Allianz\nAs a member of the worldwide Allianz Group, we use our years of local expertise, combined with global experience, to offer a wide range of products and services to our customers.', 'Motor Insurance > About Allianz\nWe are committed to continuous improvement of our products and services and strive to achieve this through the use of dedicated technical research units and the sharing of new product developments globally.

In [ ]:
response = claude_on_openai_query_engine.query("What's the product?")
print(response)
print([node.text for node in response.source_nodes])

Based on the provided context, this product disclosure statement is for a Motor Insurance policy. The document states "This Product Disclosure Statement (PDS) is an important document that contains information designed to help you make an informed decision about whether to purchase this insurance." Since it specifically references motor insurance, that appears to be the product.
['PRODUCT DISCLOSURE STATEMENT\nPreparation date: 27/01/2022.', 'Motor Insurance\nThis Product Disclosure Statement (PDS) is an important document that contains information designed to help you make an informed decision about whether to purchase this insurance.']


In [ ]:
response = claude_on_openai_query_engine.query("What's motor insurance?")
print(response)
print([node.text for node in response.source_nodes])

Unfortunately, I do not have enough context in the provided information to definitively state what motor insurance is. The context refers to "Motor Insurance" in the text, but does not explicitly define or describe what it is. Without additional details or a clear definition, I cannot reliably explain what motor insurance refers to. I can only avoid assumptions and respond based solely on the limited context provided. Please provide more specific information that clearly defines key terms if you would like me to accurately answer questions about them.
['Motor Insurance > Motor Insurance – a snapshot > What cover\nyou get for your vehicle (Not applicable if you have Third Party Property Damage cover)', 'Motor Insurance > Motor Insurance – a snapshot > What cover\nWhat we pay for (where applicable)']


In [ ]:
response = claude_on_openai_query_engine.query("What is covered?")
print(response)
print([node.text for node in response.source_nodes])

Unfortunately, I do not have enough context in the provided information to definitively state what is covered. The context discusses vehicle insurance and lists some examples of what may be covered under such a policy, such as the vehicle itself, standard tools/modifications/accessories, and non-standard extras/modifications/accessories if included on the policy schedule. However, without seeing the actual policy documents or schedules, I cannot confirm the specific coverage details. I can only reference the general information provided in the context. Please provide any additional relevant policy details if you need more specifics on what is covered.
['Motor Insurance > Motor Insurance – a snapshot > What cover\nWhat we pay for (where applicable)', 'Motor Insurance > Motor Insurance – a snapshot > What cover\n• Your vehicle (including its standard tools, modiﬁcations and accessories as supplied by the manufacturer and fitted or non-standard extras, modifications and accessories which 

### Claude LLM on Gemini Pro embedding

In [ ]:
# Create query engine using claude-2.1 model on Gemini Pro embedding
claude_on_gemini_query_engine = gemini_index.as_query_engine(llm=claude_llm)

In [ ]:
# Run query & print response

response = claude_on_gemini_query_engine.query("What is Allianz?")
print(response)
print([node.text for node in response.source_nodes])

Unfortunately I cannot directly reference or quote the provided context in my response. Based on the information given, Allianz seems to be an insurance company that is part of a larger global group. It offers a range of insurance products and services to customers, including motor insurance. The company aims to continuously improve its offerings through technical research and by sharing product developments across the global Allianz group. However, without directly citing the context, I cannot definitively state what Allianz is. I can only infer information about the company from what is provided.
['Motor Insurance > About Allianz\nAs a member of the worldwide Allianz Group, we use our years of local expertise, combined with global experience, to offer a wide range of products and services to our customers.', 'Motor Insurance > About Allianz\nWe are committed to continuous improvement of our products and services and strive to achieve this through the use of dedicated technical resear

In [ ]:
response = claude_on_gemini_query_engine.query("What's the product?")
print(response)
print([node.text for node in response.source_nodes])

Unfortunately, the context provided does not specify what the product is. It only includes a preparation date for a product disclosure statement and some definitions, but does not give any details about the actual product itself. Without more specific information about the product in the given context, I do not have enough information to determine or describe what the product is.
['PRODUCT DISCLOSURE STATEMENT\nPreparation date: 27/01/2022.', 'Definitions\nText 086 12.21']


In [ ]:
response = claude_on_gemini_query_engine.query("What's motor insurance?")
print(response)
print([node.text for node in response.source_nodes])

Unfortunately, I do not have enough context in the provided information to fully define what motor insurance is without making assumptions. The snippet discusses emergency accommodation and traveling expense coverage under a motor insurance policy, as well as a brief mention of "what we pay for" in motor insurance. However, there are no details provided on the overall definition or purpose of motor insurance as a product. To answer this query accurately, I would need more complete context defining the key aspects and coverage provisions of motor insurance policies. Without those details, I do not want to provide an incomplete or potentially inaccurate definition based solely on the limited information given. Please provide some more context around the nature and general purpose of motor insurance and I would be happy to formulate an appropriate definition for you.
['Motor Insurance > Damage\nEmergency accommodation and travelling expenses $1,500 any one incident ✔ ✘ ✘', 'Motor Insuranc

In [ ]:
response = claude_on_gemini_query_engine.query("What is covered?")
print(response)
print([node.text for node in response.source_nodes])

Unfortunately I do not have enough context to definitively state what is covered. The provided information discusses motor insurance and some details around comprehensive insurance, but does not specify what particular coverage or benefits are included. Additional details would be needed to specify what specific coverages or protections are provided under this particular motor insurance policy.
['Motor Insurance > Motor Insurance – a snapshot > What cover\nWhat we pay for (where applicable)', 'Comprehensive insurance > Accidental loss of or damage to your vehicle\nTemporary cover on replacement vehicle']


## Evaluate Response

In [25]:
# Needed for .evaluate_response() method
nest_asyncio.apply()

In [18]:
# The evaluator LLM should be more powerful than the LLM being evaluated
model_str = "gpt-4-1106-preview"
evaluator_llm = OpenAI(model=model_str, api_key=os.environ["OPENAI_API_KEY"])

In [19]:
random.seed(3)
random_20_docs = random.sample(docs, 20)

In [20]:
for doc in random_20_docs:
    print(doc)

Doc ID: 1e8247f8-ebb5-48bc-a80b-b4def3f028ef
Text: NRMA > Inspections Lifetime guarantee on authorised repairs If
we authorise and pay for a repairer to fix your vehicle, then we
provide a lifetime guarantee for the workmanship of those repairs as
long as you are the owner of your vehicle.
Doc ID: 97ac01ce-eba8-40df-ae8a-86db18a68911
Text: Allianz > Legal liability > Other beneﬁts we will pay > (This
benefit is applicable to Third Party Insurance and Third Party Fire &
Theft Insurance only) We will cover your vehicle for loss or damage
arising from an accident caused by the driver of an uninsured vehicle
up to a maximum amount of $5,000 for any one incident including the
cost of p...
Doc ID: dd83610e-1d36-4905-9de5-b725d7921ddd
Text: Allianz > Comprehensive insurance > Other beneﬁts we will pay >
Lock re-keying/re-coding We will arrange a rental car for you. If you
arrange your own rental car without our consent, we are not obliged to
pay for the rental car you arranged.
Doc ID: 99d774

In [21]:
dataset_generator = RagDatasetGenerator.from_documents(
    random_20_docs,
    llm=evaluator_llm,
    num_questions_per_chunk=1,
    show_progress=True,
)

Parsing nodes:   0%|          | 0/20 [00:00<?, ?it/s]

In [98]:
# # Generate & save the dataset as a JSON file
# rag_dataset = dataset_generator.generate_dataset_from_nodes()
# rag_dataset.save_json("pds_labelled_rag_dataset.json")

  0%|          | 0/20 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:07<00:00,  7.87s/it]


In [16]:
# Load the dataset from JSON file on disk
rag_dataset = LabelledRagDataset.from_json("pds_labelled_rag_dataset.json")

In [17]:
rag_dataset.to_pandas()

,query,reference_contexts,reference_answer,reference_answer_by,query_by
0,What condition must be met for a vehicle owner...,[NRMA > Inspections\nLifetime guarantee on aut...,The condition that must be met for a vehicle o...,ai (gpt-4-1106-preview),ai (gpt-4-1106-preview)
1,Explain the extent of coverage provided by All...,[Allianz > Legal liability > Other beneﬁts we ...,"Based on the provided context information, All...",ai (gpt-4-1106-preview),ai (gpt-4-1106-preview)
2,According to the Allianz Comprehensive insuran...,[Allianz > Comprehensive insurance > Other ben...,According to the Allianz Comprehensive insuran...,ai (gpt-4-1106-preview),ai (gpt-4-1106-preview)
3,"According to the NRMA Insurance policy, under ...","[NRMA > Insurance 3\nHowever, you may choose y...","According to the NRMA Insurance policy, a poli...",ai (gpt-4-1106-preview),ai (gpt-4-1106-preview)
4,What types of incidents are covered under the ...,[Allianz > Motor Insurance > Motor Insurance –...,"Based on the provided context information, the...",ai (gpt-4-1106-preview),ai (gpt-4-1106-preview)
5,"Explain how the ""Lock re-keying/re-coding"" ben...",[Allianz > Third Party Fire and Theft > Other ...,"Based on the context information provided, the...",ai (gpt-4-1106-preview),ai (gpt-4-1106-preview)
6,Explain how Allianz determines the initial No ...,[Allianz > Our agreement with you > Your No Cl...,According to the provided agreement terms from...,ai (gpt-4-1106-preview),ai (gpt-4-1106-preview)
7,In the event of a total loss claim under a Com...,[Allianz > Making a claim > Pay your claim as ...,In the event of a total loss claim under a Com...,ai (gpt-4-1106-preview),ai (gpt-4-1106-preview)
8,"Explain the concept of ""Additional cover for s...",[Allianz > Legal liability > Additional cover ...,"The concept of ""Additional cover for supplemen...",ai (gpt-4-1106-preview),ai (gpt-4-1106-preview)
9,"Based on the context information provided, her...",[NRMA > Comprehensive\nGeneral exclusions also...,"Based on the context information provided, her...",ai (gpt-4-1106-preview),ai (gpt-4-1106-preview)


In [38]:
faithfulness_evaluator = FaithfulnessEvaluator(llm=evaluator_llm)

In [40]:
response = openai_query_engine.query(rag_dataset[1].query)
evaluation_result = faithfulness_evaluator.evaluate_response(response=response)

print(evaluation_result.passing)
print(evaluation_result.score)
print(evaluation_result.feedback)
print(evaluation_result.response)
print([node.text for node in response.source_nodes])

True
1.0
YES
Allianz provides coverage for loss or damage to your vehicle in the event of an accident caused by the driver of an uninsured vehicle up to a maximum amount of $5,000 for any one incident. This coverage includes the cost of protection, removal, and towing as part of the benefit.
['Allianz > Legal liability > Other beneﬁts we will pay > (This benefit is applicable to Third Party Insurance and Third Party Fire & Theft Insurance only)\nWe will cover your vehicle for loss or damage arising from an accident caused by the driver of an uninsured vehicle up to a maximum amount of $5,000 for any one incident including the cost of protection, removal and towing.', 'Allianz > Comprehensive insurance > Accidental loss of or damage to your vehicle\nWe will cover you for accidental loss of (including theft) or any other type of accidental damage that happens to your vehicle during the period of insurance, subject to the other terms of this section and the policy (including those relatin

In [41]:
response = gemini_query_engine.query(rag_dataset[1].query)
eval_result = faithfulness_evaluator.evaluate_response(response=response)

print(evaluation_result.passing)
print(evaluation_result.score)
print(evaluation_result.feedback)
print(evaluation_result.response)
print([node.text for node in response.source_nodes])

True
1.0
YES
Allianz provides coverage for loss or damage to your vehicle in the event of an accident caused by the driver of an uninsured vehicle up to a maximum amount of $5,000 for any one incident. This coverage includes the cost of protection, removal, and towing as part of the benefit.
['Allianz > Legal liability > Other beneﬁts we will pay > (This benefit is applicable to Third Party Insurance and Third Party Fire & Theft Insurance only)\nWe will cover your vehicle for loss or damage arising from an accident caused by the driver of an uninsured vehicle up to a maximum amount of $5,000 for any one incident including the cost of protection, removal and towing.', 'Allianz > Comprehensive insurance > Other beneﬁts we will pay\nUnless we have stated differently under one of the other benefits listed below, the benefit will only apply where we have accepted your claim for accidental loss of or accidental damage to your vehicle (called a covered accident) under the policy and any payme

In [22]:
deepeval_faithfulness_evaluator = DeepEvalFaithfulnessEvaluator(model=model_str)

In [26]:
query = rag_dataset[0].query
response = openai_query_engine.query(query)
evaluation_result = deepeval_faithfulness_evaluator.evaluate_response(
    query=query,
    response=response,
)

Output()

In [30]:
print(evaluation_result.passing)
print(evaluation_result.score)
print(evaluation_result.feedback)
print(evaluation_result.response)

True
1.0
The score is 1.00 because there are no contradictions between the actual output and the retrieval context, indicating perfect alignment and faithfulness. Keep up the good work!
The vehicle owner must still be the owner of the vehicle for them to receive a lifetime guarantee on the workmanship of repairs authorized and paid for by NRMA.


In [31]:
query = rag_dataset[0].query
response = gemini_query_engine.query(query)
evaluation_result = deepeval_faithfulness_evaluator.evaluate_response(
    query=query,
    response=response,
)

Output()

In [36]:
print(evaluation_result.passing)
print(evaluation_result.score)
print(evaluation_result.feedback)
print(evaluation_result.response)
print([node.text for node in response.source_nodes])

True
1.0
The score is 1.00 because there are no contradictions listed, indicating that the actual output aligns perfectly with the retrieval context. Great job on maintaining accuracy!
The vehicle owner must remain the owner of the vehicle for the lifetime of the guarantee.
['NRMA > Inspections\nLifetime guarantee on authorised repairs If we authorise and pay for a repairer to fix your vehicle, then we provide a lifetime guarantee for the workmanship of those repairs as long as you are the owner of your vehicle.', 'NRMA > Inspections\nHowever, you can’t claim under our lifetime guarantee if: you sold your vehicle – that is, you are no longer the owner of your vehicle • you (or your agent) authorised repairs to your vehicle without our prior written consent you arrange repairs after we cash settle your claim – that is, we do not authorise repairs there is loss or damage to or failure of any electrical or mechanical part or component, or there is deterioration or wear and tear caused:\n–